Based on: https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [7]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [9]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(input)
out

tensor([[[[ 1.9446, -0.3029, -0.7139,  ..., -0.0103, -1.4306, -0.1855],
          [-0.0266, -0.3775,  0.5732,  ..., -0.5679, -0.0884, -0.2014],
          [-0.6116, -0.6399,  1.3589,  ...,  1.2021, -1.0893,  0.1091],
          ...,
          [-0.2402,  0.2858,  0.6680,  ..., -2.1725,  1.5914,  1.3850],
          [-0.4651, -0.9497, -0.6317,  ..., -0.8820,  0.0935, -0.6813],
          [ 0.9993, -2.0373, -1.1197,  ..., -2.5142,  0.6452, -0.8217]]]])


tensor([[-0.0656, -0.0168,  0.0149,  0.0228,  0.1034,  0.0289,  0.2223,  0.0309,
          0.0010,  0.1483]], grad_fn=<AddmmBackward>)

#### Loss Function

In [12]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5216, grad_fn=<MseLossBackward>)


In [13]:
loss.grad_fn

#### Calculate Backprop

We just need to acll `loss.backward()`

In [14]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 6.0934e-04, -2.3730e-03, -1.6088e-03,  1.3720e-05, -4.8454e-03,
         2.2587e-04])


#### Optimization: Update the weights

Now with the Gradients we can update the weights with SGD

In [17]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [18]:
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

In [19]:
net.conv1.bias.grad

tensor([-0.0021, -0.0047, -0.0036, -0.0032, -0.0061, -0.0001])